# Reading, cleaning

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_squared_log_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.datasets import make_regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression, RFECV, SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA

import time

from sklearn import set_config
import warnings
from google.colab import files

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.3f}'.format

url = '' # link to a csv-file on google drive
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
data_original = pd.read_csv(path)

# drop 'Id' as it contains unique values only
X = data_original.drop(columns=['Id'])

# split to X and y
y = X.pop('SalePrice')

# split to train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Preprocessing pipeline (impute + convert to num + scale)

In [ ]:
X_cat = X.select_dtypes(exclude='number').copy()
X_num = X.select_dtypes(include='number').copy()

# numerical brunch
num_constant_cols = ['GarageYrBlt']
num_mean_median_cols = list(X_num.drop(columns = num_constant_cols)) # list of indexes for ColumnTransformer

# categorical brunch
cat_nan_frequent_list = ['MasVnrType', 'Electrical', 'Utilities',
                         'ExterQual', 'HeatingQC', 'ExterCond',
                         'KitchenQual', 'Functional', 'LandSlope',
                         'CentralAir']

cat_nan_frequent_cols = X_cat.columns.get_indexer(cat_nan_frequent_list) # list of indexes for ColumnTransformer
cat_nan_fill_cols = list(X_cat.drop(X_cat.columns[cat_nan_frequent_cols], axis = 1)) # list of indexes for ColumnTransformer

#### strategy 'most_frequent' seems to work incorrectly, therefore 'N_A' is needed in columns like ExterQual below)
categorical_imputer = ColumnTransformer(
    transformers=[
        ('cat_nan_frequent', SimpleImputer(strategy='most_frequent'), cat_nan_frequent_cols),
        ('cat_nan_fill', SimpleImputer(strategy='constant', fill_value='N_A'), cat_nan_fill_cols)
        ]
)

cat_ordinal_list = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                    'BsmtExposure', 'HeatingQC', 'KitchenQual', 'Functional',
                    'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC',
                    'Fence', 'Utilities', 'LandSlope', 'BsmtFinType1',
                    'BsmtFinType2', 'CentralAir', 'GarageFinish']

cat_ordinal_cols = X_cat.columns.get_indexer(cat_ordinal_list)

# "N_A" everywhere here helped, though imputer above should avoid NaNs to appear
ordinal_features_categories = [['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],  # ExterQual
                               ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],  # ExterCond
                               ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],  # BsmtQual
                               ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],  # BsmtCond
                               ['Gd', 'Av', 'Mn', 'No', 'N_A'],        # BsmtExposure
                               ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],  # HeatingQC
                               ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],  # KitchenQual
                               ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 
                                'Maj2','Sev', 'Sal', 'N_A'],           # Functional
                               ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],  # FireplaceQu
                               ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],  # GarageQual
                               ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],  # GarageCond
                               ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'N_A'],  # PoolQC
                               ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'N_A'],  # Fence
                               ['AllPub', 'NoSewr', 'NoSeWa', 'ELO', 'N_A'],  # Utilities
                               ['Gtl', 'Mod', 'Sev', 'N_A'],           # LandSlope
                               ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'N_A'],  # BsmtFinType1
                               ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'N_A'],   # BsmtFinType2
                               ['Y', 'N', 'N_A'],                      # CentralAir
                               ['Fin', 'RFn', 'Unf', 'N_A']]           # GarageFinish

cat_onehot_list = list(X_cat.drop(X_cat.columns[cat_ordinal_cols], axis = 1))
cat_onehot_cols = X_cat.columns.get_indexer(cat_onehot_list)

categorical_encoder = ColumnTransformer(
    transformers=[
        ('cat_ordinal',
         OrdinalEncoder(categories = ordinal_features_categories),
         cat_ordinal_cols),
        ('cat_onehot',
         OneHotEncoder(handle_unknown='ignore',
                       drop='first',), # this parameter reduce feature number (after full_preprocessor) from 229 to 205
         cat_onehot_cols)
    ]
)

cat_pipe = make_pipeline(SimpleImputer(strategy='constant', fill_value='N_A'),
                         categorical_encoder)

# preprocessing pipe for all columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num_pipe', SimpleImputer(), X_num.columns),
        ('cat_pipe', cat_pipe, X_cat.columns),
    ]
)

full_preprocessor = make_pipeline(preprocessor, MinMaxScaler())# .set_output(transform="pandas")

set_config(display='diagram')
full_preprocessor


# RandomForestRegressor - parameter search with GridSearchCV

In [ ]:
# define parameter grid
start_time = time.time()

rf_param_grid = {
    'randomforestregressor__n_estimators': range(130, 180, 10),  # Number of trees in the forest
    'randomforestregressor__max_depth': range(12, 16, 1),        # Maximum depth of each decision tree
    'randomforestregressor__min_samples_split': range(1, 7, 1),  # Minimum number of samples required to split an internal node
    }


# define GridSearchCV
rf_search = GridSearchCV(rf_pipeline,
                         param_grid=rf_param_grid,
                         cv=5,
                         scoring='neg_mean_squared_log_error',
                         n_jobs=-1,
                         verbose=1
                         # error_score='raise'
                         )

rf_search.fit(X_train, y_train)

rf_predictions_search = rf_search.predict(X_test)

end_time = time.time()
time_taken = end_time - start_time

print(f'The best average score in GridSearchCV: {rf_search.best_score_}')
print(f'The best parameters in GridSearchCV: {rf_search.best_params_}')
print(f'The test MSLE: {mean_squared_log_error(rf_predictions_search, y_test)}')
print("--- %s seconds ---" % (time_taken))


# RandomForestRegressor - manual parameter set up

In [ ]:
rf_pipeline = make_pipeline(full_preprocessor,
                            RandomForestRegressor(n_estimators=130,
                                                  max_depth=15,
                                                  min_samples_split=4))

rf_pipeline.fit(X_train, y_train)

rf_predictions = rf_pipeline.predict(X_test)

print(f'The test MSLE: {mean_squared_log_error(rf_predictions, y_test)}')

# TEST DATASET FROM KAGGLE.COM

In [ ]:
url_prod = '' # link to a csv-file on google drive
path_prod = 'https://drive.google.com/uc?export=download&id='+url_prod.split('/')[-2]
data_prod_original = pd.read_csv(path_prod)

In [ ]:
X_prod = data_prod_original.copy()
df_predict = X_prod.pop('Id').to_frame()

y_predict = rf_pipeline.predict(X_prod)

df_predict['SalePrice'] = y_predict
df_predict

## Download the datframe with price prediction

In [ ]:
output_path = 'sample_submission.csv'
df_predict.to_csv(output_path, index=False)
files.download('sample_submission.csv')